In [4]:
import copy
import numpy as np
import pandas as pd

inspection_df = copy.deepcopy(pd.read_csv('inspections_final.csv'))
business_df = copy.deepcopy(pd.read_csv('business_final.csv',index_col=0))
review_df = copy.deepcopy(pd.read_csv('review_final.csv'))

In [3]:
review_df.tail(2)

,id,business_id,user_id,stars,date,text,useful,funny,cool
809048,___Dkf_2zULN6-kdhckwcw,8K3CRM4COa0SSBEvli0fJQ,vvxIVr_OR6sHBzGPG9dRXg,4,05/16/2016 00:00:00,I'm a big fan of Mediterranean food.\nLove fre...,3.0,1.0,3.0
809049,___tiHHnNV42l-Bo8edxIw,PgJgpW4TKQ8IoALH2Yq-zA,NQ22CXR7Z-JpoNJFszJG8w,4,08/21/2016 00:00:00,Another great buffet on the trip. Great select...,0.0,0.0,0.0


In [5]:
review_df.shape

(809050, 9)

In [6]:
class Review_Info:
    def __init__(self,df):
        self.nparr = df.as_matrix()
        self.text = {}
        self.keywords = {} 
        self.star = {}
        self.has_keyword_count = {}  # has_keywords = {'business_id': int (count of reviews with keyword, helpful vote +1)}
        self.count_one_star = {}
    def convert_dict(self):
        for row in range(len(self.nparr)):
            current_id = self.nparr[row][1]
            current_text = self.nparr[row][5]
            current_star = self.nparr[row][3]
            helpful = self.nparr[row][6]
            self.add_to_dict(current_id,current_star,self.star)        # current review star
            if current_id not in self.count_one_star:
                self.count_one_star[current_id] = 0
            if int(current_star) == 1:
                self.count_one_star[current_id] += 1                   # count 1stars for current restaurant (unweighted)
            if not pd.isnull([current_text]):
                self.add_to_dict(current_id,current_text,self.text)    # review text
                self.count_keywords(current_id,current_text,helpful)   # keywords
        print("convert_dict() done")
    def add_to_dict(self,key_to_add,val_to_add,d):
        if key_to_add not in d:
            d[key_to_add] = []
        d[key_to_add].append(val_to_add)
    def count_keywords(self,current_id,current_text,helpful):
        keyword_list = ['sick','dirty','disgusting','vomit','threw up','food poison','diarrhea',
                        'filthy', 'gross', 'throw up', 'throwing up','indigestion', 'nausea', 'nauseous',
                        'upset stomach', 'the runs', ' bug ',' rat ', 'mice', 'mouse', 'expired', 
                        'listeria', 'contaminate', 'bacteria', 'salmonella', 'e coli', 'infest', 'retch', 
                        ' gag ', 'stench', 'e-coli', 'tainted']
        has_keyword = False
        if current_id not in self.keywords:    # count each keyword
            self.keywords[current_id] = {}
        for word in keyword_list:
            if word not in self.keywords[current_id]:
                self.keywords[current_id][word] = 0
            if word in current_text:
                self.keywords[current_id][word] += 1
                has_keyword = True
        if has_keyword:              # count reviews with >= 1 keyword (1 + number of helpful votes)
            if current_id not in self.has_keyword_count:
                self.has_keyword_count[current_id] = 0
            self.has_keyword_count[current_id] += (helpful + 1)
    def nparr(self):
        return self._nparr
    def keywords(self):               # keywords={'business_id1':{'sick': 2,'dirty': 2},{'business_id2'}: {'sick': 2,'dirty': 2}}
        return self._keywords
    def text(self):                   # text['business_id'] = ["One of my absolute favorite",...] list of str
        return self._text
    def star(self):                   # star['business_id'] = [1,5]   list of int
        return self._star
    def count_one_star(self):         # count_one-star = {'business_id': int}
        return self._count_one_star()

review = Review_Info(review_df)
review.convert_dict()

convert_dict() done


In [7]:
print("Number of unqiue businesses with keywords recorded:",len(review.keywords))
print("Sample keyword dict:\n", review.keywords['PgJgpW4TKQ8IoALH2Yq-zA'])

Number of unqiue businesses with keywords recorded: 9771
Sample keyword dict:
 {'sick': 12, 'dirty': 20, 'disgusting': 10, 'vomit': 1, 'threw up': 0, 'food poison': 6, 'diarrhea': 0, 'filthy': 0, 'gross': 15, 'throw up': 2, 'throwing up': 1, 'indigestion': 0, 'nausea': 1, 'nauseous': 1, 'upset stomach': 2, 'the runs': 0, ' bug ': 1, ' rat ': 0, 'mice': 0, 'mouse': 1, 'expired': 0, 'listeria': 0, 'contaminate': 0, 'bacteria': 0, 'salmonella': 0, 'e coli': 0, 'infest': 0, 'retch': 2, ' gag ': 0, 'stench': 0, 'e-coli': 0, 'tainted': 0}


In [8]:
print("Number of businesses with reviews:", len(review.text))
print("Sample: count of reviews for one business (all times):",len(review.text['PgJgpW4TKQ8IoALH2Yq-zA']))
# print("Sample review:", review.text['PgJgpW4TKQ8IoALH2Yq-zA'][0])
print("Number of businesses with stars:",len(review.star))
print("Sample business mean stars:",np.mean(review.star['PgJgpW4TKQ8IoALH2Yq-zA']))

Number of businesses with reviews: 9771
Sample: count of reviews for one business (all times): 1034
Number of businesses with stars: 9772
Sample business mean stars: 2.82688588008


In [12]:
review.text.keys()

dict_keys(['R1PQEK6qvrZVC9qcWfKvDA', '3kdSl5mo9dWC4clrQjEDGg', '4RoTEeqB_MNn6yaqZmlZHg', 'TTDMJetAQKfxVzKZy4Z_2Q', 'WYw3Uf56DT5IwpaLNnCH5Q', 'hihud--QRriCYZw1zZvW4g', '16tWXenJ1wTqugXJU7yiZg', 'J5ylw6DXGmQ6hM06Ne3XHw', 'y7Js-07RF8d3N_AEtaw2VQ', 'RESDUcs7fIiihp38-d6_6g', 'xmvvkp5-O29l3EQQFEf3YQ', 'IMLrj2klosTFvPRLv56cng', 'rJYZyjWH_GCMv-JZdykv6g', 'eEnNw3_hBvxcFHyr23kAuA', 'tH0wF3EPPgAY14lnmK1_Sg', '07gh-AImcEcWQ2bew-jprg', 'dM8Yp8StA1NdusK5Ta_j-g', '1MrSwj8i2VEBHdp3IsWCpQ', 'gH0E4kqHaWjxoRtin7dmKA', '4ubvNMCunbZt0NrVqySXPw', 'bffQssCZ-KpA_PPIu6Vysg', 'pdgzBiQBs8HGUBJzXe9j6w', '8olG8J91NG_QU92WHG5QfA', 'zBD6zc8YU-TA0_KJmN1tpA', 'y4ETe3DkcIjjr_eyeg5U5g', 'X-WXPGYuuknAjZkpYGeRiw', 'gTlDDzDEHyDQ6iwjNhpI6A', 'SUH0cMrRsysIu2hKqQQKBA', 'ugLqbAvBdRDc-gS4hpslXw', 'LYWbyu1EQM2o5hwHG5_Ldg', 'KwKkNTxYGyo4H6pKZ6usSA', 'JpgVl3d20CMRNjf1DVnzGA', '-3zffZUHoY8bQjGfPSoBKQ', 'upxEHDxnEPwuts9xY0ylng', 'xtYiHTmunjfCN2sUaQxBjA', 'IkI9yPjx-MRYh-Jh-LqIiQ', 'KjjNv4YFzBFnsBh2ncH1Iw', 'pRmb8_tAP5y4YPt_viL07g', '

In [9]:
review.count_one_star['hZAxcfDnLOkFY6G5ofI2RQ']

5

In [26]:
business_df.shape

(9771, 10)

In [10]:
business_df.head(2)

,business_id,name,neighborhood,address,postal_code,latitude,longitude,stars,review_count,categories
Column,,,,,,,,,,
6,kCoE3jvEtg6UVz5SOD3GVw,BDJ Realty,Summerlin,2620 Regatta Dr,89128,36.2074,-115.268460,4.0,5,Real Estate Services;Real Estate;Home Services...
26,VBHEsoXQb2AQ76J9l8h1uQ,Alfredo's Jewelry,Southeast,5775 S Eastern,89119,36.0851,-115.119421,4.5,23,Shopping;Jewelry;Watch Repair;Local Services


In [11]:
class Business_Info:
    def __init__(self,df):
        self.nparr = df.as_matrix()
        self.address_id = {}        
        self.id_address = {}
        self.name_id = {}
        self.id_name = {}
        self.star = {} 
        self.review_count = {}
    def convert_dict(self):
        for row in range(len(self.nparr)):
            business_id = self.nparr[row][0]
            name = self.nparr[row][1]
            address = self.nparr[row][3]
            star = self.nparr[row][7]
            review_ct = self.nparr[row][8]
            self.add_to_dict(address,business_id,self.address_id)
            self.add_to_dict(business_id,address,self.id_address)
            self.add_to_dict(business_id,name,self.id_name)
            self.add_to_dict(business_id,star,self.star)
            self.add_to_dict(name,business_id,self.name_id)
            self.add_to_dict(business_id,review_ct,self.review_count)
        print("convert_dict() done")
    def add_to_dict(self,key_to_add,val_to_add,d):
        if not pd.isnull([key_to_add,val_to_add]).any():
            if key_to_add not in d:
                d[key_to_add] = []
            d[key_to_add].append(val_to_add)
    def nparr(self):
        return self._nparr
    def address_id(self):          # address_id = {'address':'business_id'}
        return self._address_id
    def id_address(self):          # id_address = {'id': 'address'}
        return self._id_address
    def star(self):                # star = {'business_id': float}
        return self._star
    def review_count(self):        # review_count = {'business_id': int}
        return self._review_count
    def id_name(self):
        return self._id_name
    def name_id(self):             # name = {'business_id': 'name'}
        return self._name_id
    
business = Business_Info(business_df)
business.convert_dict()

convert_dict() done


In [28]:
print("Number of unique addresses (address - business_id dict):",len(business.address_id))
len(business.star) == len(business.review_count) == len(business.id_name)

Number of unique addresses (address - business_id dict): 2894


True

In [29]:
inspection_df.shape

(92500, 17)

In [30]:
inspection_df.tail(2)

,Permit Number,Restaurant Name,Location Name,Category Name,Address,Zip,Current Demerits,Current Grade,Inspection Date,Inspection Time,Employee ID,Inspection Type,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations
92498,PR0020055,Staybridge Suites - Kitchen - Buffet,Staybridge Suites,Buffet,5735 Dean Martin Dr,89118,9,A,02/03/2017 12:00:00 AM,02/03/2017 09:30:00 AM,EE7001214,Routine Inspection,9,A,A,'A' Grade,"213,215,2909,2928,2930"
92499,PR0013865,Vickie's Diner,Vickie's Diner,Restaurant,1700 S Las Vegas Blvd,89104,36,C,05/22/2017 12:00:00 AM,05/22/2017 11:10:00 AM,EE7001083,Re-inspection,0,A,A,'A' Grade,NaN


In [12]:
class Inspection_Info:
    def __init__(self,df):
        self.nparr = df.as_matrix()
        self.address = {}
        self.name = {}
        self.demerit = {} 
        self.grade = {}
    def convert_dict(self):
        for row in range(len(self.nparr)):
            name = self.nparr[row][2]        # 'Location Name'
            address = self.nparr[row][4]
            demerit = self.nparr[row][12]    # 'Inspection Demerits'
            grade = self.nparr[row][13]      # 'Inspection Grade'
            self.add_to_dict(address,name,self.address)
            self.add_to_dict(address,demerit,self.demerit)
            self.add_to_dict(address,grade,self.grade)
            self.add_to_dict(name,address,self.name)
        print("conver_dict() done")
    def add_to_dict(self,key_to_add,val_to_add,d):
        if not pd.isnull([key_to_add,val_to_add]).any():
            if key_to_add not in d:
                d[key_to_add] = []
            d[key_to_add].append(val_to_add)
    def nparr(self):
        return self._nparr
    def address(self):    # address = {'address': 'name'}
        return self._name
    def demerit(self):    # demerit = {'address': int}
        return self._demerit
    def grade(self):      # grade = {'address': char}
        return self._grade
    def name(self):       # name = {'name': 'address'}
        return self._name

inspection = Inspection_Info(inspection_df)
inspection.convert_dict()

conver_dict() done


In [36]:
print("Number of unique addresses in inspection:",len(inspection.address.keys()))

Number of unique addresses in inspection: 2894


In [37]:
inspection.grade.keys() == inspection.demerit.keys() == inspection.grade.keys()

True

In [38]:
len(inspection.name)

4961

In [13]:
# inspection date and review date are left out for now
import difflib
def aggregate(business, inspection, review):
    op = {}
    ''' iterate thru inspection (skip restaurants w/o inspection) '''
    for inspection_address, inspection_name_list in inspection.address.items():
        for inspection_name in inspection_name_list:
            for business_id in business.address_id[inspection_address]:
                yelp_name = business.id_name[business_id][0]
                if difflib.SequenceMatcher(None,yelp_name.lower(),inspection_name.lower()).ratio() < 0.1:
                    continue              # skip record with same address but different store names
                else:                     # if unique name create row
                    if business_id not in op:
                        op[business_id] = {}
                    op[business_id]['address'] = business.id_address[business_id][0]
                    op[business_id]['name'] = yelp_name
                    op[business_id]['yelp_star'] = business.star[business_id][0]
                    op[business_id]['mean_star'] = np.mean(review.star[business_id])
                    op[business_id]['review_count'] = len(review.text[business_id])
                    op[business_id]['demerit'] = inspection.demerit[inspection_address]
                    op[business_id]['grade'] = inspection.grade[inspection_address]
                    op[business_id]['keyword_count'] = review.keywords[business_id]
                    # inspection fail count 
                    inspection_fail_ct = 0
                    for grade in inspection.grade[inspection_address]:
                        if grade != 'A':
                            inspection_fail_ct += 1
                    op[business_id]['inspection_fail_count'] = inspection_fail_ct
                    op[business_id]['inspection_fail_indicator'] = 1 if inspection_fail_ct > 0 else 0
                    # one star proportion
                    one_star_ct = review.count_one_star[business_id]
                    total_review_ct = len(review.star[business_id])
                    op[business_id]['one_star_count'] = one_star_ct
                    op[business_id]['proportion_of_one_star'] = one_star_ct / total_review_ct
    return op

op = aggregate(business,inspection,review)

In [41]:
len(op.keys())

9640

In [14]:
op['3iQ4H1WRqg1EvSZSRLeb-A']['one_star_count']

1

In [15]:
aggregate_df = pd.DataFrame.from_dict(op, orient='index')

In [16]:
aggregate_df.head(5)

,address,name,yelp_star,mean_star,review_count,demerit,grade,keyword_count,inspection_fail_count,inspection_fail_indicator,one_star_count,proportion_of_one_star
-0d4L4U3vKnTT0MEgyX_WA,6620 W Flamingo Rd,Pick N Puff,5.0,4.900000,10,"[6, 8, 0, 6, 6, 7, 10, 10, 8, 3, 7, 0, 7, 14, ...","[A, A, A, A, A, A, A, nan, A, A, A, A, A, B, A...","{'sick': 0, 'dirty': 0, 'disgusting': 0, 'vomi...",6,1,0,0.000000
-1uVyyy3UXNA9WoPucNniA,8826 S Eastern Ave,La Tapatia Market,5.0,4.933333,15,"[8, 9, 8, 20, 3, 6, 0, 17, 6, 3, 18, 1, 10, 8,...","[A, A, A, B, A, A, A, B, A, A, B, A, A, A, A, ...","{'sick': 0, 'dirty': 0, 'disgusting': 0, 'vomi...",21,1,0,0.000000
-1vfRrlnNnNJ5boOVghMPA,6630 N Durango Dr,Red Ginseng Narita Sushi & BBQ,3.0,3.226667,75,"[0, 0, 0, 1, 3, 6, 1, 8, 5, 7, 10, 10, 3, 6, 8...","[A, A, A, A, A, A, A, A, A, A, A, B, A, A, A, ...","{'sick': 1, 'dirty': 2, 'disgusting': 1, 'vomi...",1,1,15,0.200000
-2S3tYZEy-JbQonqdIsfqg,4001 S Decatur Blvd,Epic Thrift,3.5,3.588235,17,"[9, 3, 0, 6, 8, 19, 10, 9, 19, 0, 0, 6, 2, 6, ...","[A, A, A, A, A, B, A, A, B, A, A, A, A, A, A, ...","{'sick': 0, 'dirty': 1, 'disgusting': 0, 'vomi...",19,1,2,0.117647
-2_l2p8bdI91G3-EIjSikg,3200 S Las Vegas Blvd,Adore Organic Innovation,1.5,1.470588,17,"[6, 8, 0, 0, 18, 0, 9, 6, 5, 0, 0, 5, 8, 8, 10...","[A, A, A, A, B, A, A, A, A, A, A, A, B, A, A, ...","{'sick': 0, 'dirty': 0, 'disgusting': 0, 'vomi...",138,1,15,0.882353


In [17]:
aggregate_df.shape

(9640, 12)

In [18]:
aggregate_df.to_csv(path_or_buf='aggregate.csv',
                    header=['address','name','yelp_star','mean_star','review_count','demerit',
                     'grade','keyword_count','inspection_fail_count','inspection_fail_indicator',
                     'one_star_count','prop_one_star'],index=True)
print("done")

done


In [ ]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
def linear_reg(df):
    X = df['mean_star'].as_matrix().reshape((9640,1))
    Y = df['yelp_star'].as_matrix().reshape((9640,1))
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y)
    linear_reg = LinearRegression().fit(X_train,Y_train)
    print(-1 * cross_val_score(linear_reg,X_train,Y_train,cv=5,scoring='neg_mean_squared_error'))
linear_reg(aggregate_df)    